In [8]:

# standard library imports
import git, os, sys

git_repo = git.Repo(os.getcwd(), search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")
os.chdir(f'{git_root}/src')
sys.path.append(os.path.abspath(os.path.join(f'{git_root}/src')))
print(f'Changed working directory to {os.getcwd()}')

# local imports
from fight_stat_helpers import *
from data_handler import DataHandler

Changed working directory to C:\Users\Alex\OneDrive\Documents\GitHub\UFC_Prediction_2022\src


# NEW FEATURES

- Dominance Score for a single fight: (total score of fighter based on events in the fight) - (total score of opponent based on events in the fight)
- Average Dominance over time scales
- Fighter Scores
- Fight Math
- Fighter Score increments based on closeness of fights with opponents with high scores 

# Other things to try
- with or without regularization
- with or without scaling
- with or without sum features (seems like only age needs sum features)
- with or without squared features (seems like only age needs squared features)
- Decide whether to include SDEC in bullshit or not
- Do linear reg on draws and SDEC and see how this compares to log reg plane (hypothesis: they are the same plane since close fights should be near the plane boundary)

In [9]:
dh = DataHandler()
# get reported derived doubled data
ufc_fights_reported_derived_doubled = dh.get('ufc_fights_reported_derived_doubled')

# Verify FIGHT MATH Stats

In [3]:
ufc_fights_reported_derived_doubled[['all_fight_math']].sum()

all_fight_math    1837.0
dtype: float64

In [18]:
# only take fights in the last 10 years
ufc_fights_reported_derived_doubled['date'] = pd.to_datetime(ufc_fights_reported_derived_doubled['date'])
date_10_years_ago = pd.Timestamp.now() - pd.DateOffset(years=10)
ufc_fights_reported_derived_doubled = ufc_fights_reported_derived_doubled[ufc_fights_reported_derived_doubled['date'] >= date_10_years_ago]

In [19]:
ufc_fights_reported_derived_doubled

,date,fighter,opponent,result,method,division,age,height,reach,stance,...,l3y_offensive_grappling_score,l5y_offensive_grappling_score,all_defensive_grappling_loss,l1y_defensive_grappling_loss,l3y_defensive_grappling_loss,l5y_defensive_grappling_loss,all_overall_fighter_score,l1y_overall_fighter_score,l3y_overall_fighter_score,l5y_overall_fighter_score
6670,2015-08-08,Anthony Christodoulou,Scott Holtzman,L,SUB,Lightweight,28.024641,69.0,NaN,Orthodox,...,0.047337,0.047337,1.213018,1.213018,1.213018,1.213018,-74.508876,-74.508876,-74.508876,-74.508876
6671,2015-08-08,Scott Holtzman,Anthony Christodoulou,W,SUB,Lightweight,31.854894,69.0,69.0,Orthodox,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6672,2015-08-08,Roman Salazar,Marlon Vera,L,SUB,Bantamweight,27.460643,67.0,NaN,Orthodox,...,0.219554,0.219554,3.476844,3.476844,3.476844,3.476844,-14.519726,-14.519726,-14.519726,-14.519726
6673,2015-08-08,Marlon Vera,Roman Salazar,W,SUB,Bantamweight,22.680356,68.0,70.0,Switch,...,1.351111,1.351111,1.271111,1.271111,1.271111,1.271111,-17.920000,-17.920000,-17.920000,-17.920000
6674,2015-08-08,Chris Dempsey,Jonathan Wilson,L,KO/TKO,Light Heavyweight,27.915127,70.0,72.0,Orthodox,...,0.780915,0.780915,0.786758,0.793333,0.786758,0.786758,-14.047712,8.431111,-14.047712,-14.047712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16529,2025-08-02,Esteban Ribovics,Elves Brener,W,U-DEC,Lightweight,29.264887,70.0,69.0,Orthodox,...,0.215012,0.215012,0.904592,0.144444,0.904592,0.904592,50.947759,44.888889,50.947759,50.947759
16530,2025-08-02,Mateusz Rebecki,Chris Duncan,L,U-DEC,Lightweight,32.744695,67.0,66.0,Southpaw,...,4.187406,4.187406,0.525937,0.562222,0.525937,0.525937,22.003298,10.433333,22.003298,22.003298
16531,2025-08-02,Chris Duncan,Mateusz Rebecki,W,U-DEC,Lightweight,32.229979,70.0,71.0,Orthodox,...,7.140045,7.140045,3.324754,0.627717,3.324754,3.324754,28.278577,-9.771739,28.278577,28.278577
16532,2025-08-02,HyunSung Park,Tatsuro Taira,L,SUB,Flyweight,29.744011,67.0,66.0,Orthodox,...,6.871274,6.871274,0.230352,0.000000,0.230352,0.230352,38.405149,69.643836,38.405149,38.405149


In [20]:
ufc_fights_predictive_flattened_diffs = dh.make_ufc_fights_predictive_flattened_diffs(ufc_fights_reported_derived_doubled)
ufc_fights_predictive_flattened_diffs = dh.clean_ufc_fights_for_winner_prediction(ufc_fights_predictive_flattened_diffs)

In [21]:
ufc_fights_predictive_flattened_diffs

,fighter,opponent,result,method,division,age_diff,height_diff,reach_diff,all_wins_diff,l1y_wins_diff,...,l5y_offensive_grappling_score_diff,all_defensive_grappling_loss_diff,l1y_defensive_grappling_loss_diff,l3y_defensive_grappling_loss_diff,l5y_defensive_grappling_loss_diff,all_overall_fighter_score_diff,l1y_overall_fighter_score_diff,l3y_overall_fighter_score_diff,l5y_overall_fighter_score_diff,age_sum
4,Chris Dempsey,Jonathan Wilson,0,KO/TKO,Light Heavyweight,-0.145106,-4.0,-3.0,1.0,1.0,...,0.780915,0.786758,0.793333,0.786758,0.786758,-14.047712,8.431111,-14.047712,-14.047712,55.975359
8,Dustin Ortiz,Willie Gates,1,KO/TKO,Flyweight,-1.927447,-3.0,-7.0,2.0,-1.0,...,0.888132,-3.640309,-4.028804,-3.640309,-3.640309,32.696678,-35.959590,32.696678,32.696678,55.162218
10,Chris Camozzi,Tom Watson,1,DEC,Middleweight,-4.355921,1.0,2.0,4.0,-1.0,...,3.036812,8.436528,3.218139,5.455113,8.445653,21.635663,41.571406,19.575704,18.427454,61.785079
12,Uriah Hall,Oluwale Bamgbose,1,KO/TKO,Middleweight,3.008898,1.0,1.0,3.0,1.0,...,0.553801,0.640272,0.488809,0.640272,0.640272,8.434783,7.829006,8.434783,8.434783,59.030801
14,Ray Borg,Geane Herrera,1,DEC,Flyweight,-3.189596,-1.0,-3.0,2.0,1.0,...,8.246001,0.674021,0.190722,0.674021,0.674021,-9.523442,22.528351,-9.523442,-9.523442,47.208761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9854,Neil Magny,Elizeu Zaleski dos Santos,1,KO/TKO,Welterweight,-0.643395,4.0,7.0,11.0,-1.0,...,3.098993,15.366362,1.244329,3.160463,6.265374,5.574709,-11.560400,-1.549324,-14.147277,76.640657
9856,Karol Rosa,Nora Cornolle,1,DEC,Women's Bantamweight,-5.065024,-2.0,0.0,4.0,0.0,...,-2.824357,-0.694178,-0.184018,-0.715660,-0.612203,6.318019,10.398957,-11.965348,-3.380163,66.245038
9858,Esteban Ribovics,Elves Brener,1,DEC,Lightweight,1.418207,0.0,-3.0,0.0,1.0,...,-0.253515,0.119151,-0.068791,0.119151,0.119151,66.839385,113.577614,66.839385,66.839385,57.111567
9860,Mateusz Rebecki,Chris Duncan,0,DEC,Lightweight,0.514716,-3.0,-5.0,0.0,-1.0,...,-2.952639,-2.798817,-0.065495,-2.798817,-2.798817,-6.275278,20.205072,-6.275278,-6.275278,64.974675


In [22]:
# show division counts
print(ufc_fights_predictive_flattened_diffs['division'].value_counts())

division
Lightweight              609
Welterweight             591
Featherweight            496
Middleweight             473
Bantamweight             453
Light Heavyweight        328
Heavyweight              327
Women's Strawweight      256
Flyweight                241
Women's Flyweight        222
Women's Bantamweight     153
Catch Weight              52
Women's Featherweight     26
Name: count, dtype: int64


In [23]:
# drop the 'fighter_result' column and opponent_result column as it is not needed for the model and also fighter and opponent columns
X = ufc_fights_predictive_flattened_diffs.drop(columns=['fighter', 'opponent', 'method', 'division'])
# KEEP result in X as this is what model_score is expecting for better or for worse
y = X['result']

In [24]:
X

,result,age_diff,height_diff,reach_diff,all_wins_diff,l1y_wins_diff,l3y_wins_diff,l5y_wins_diff,all_wins_ko_diff,l1y_wins_ko_diff,...,l5y_offensive_grappling_score_diff,all_defensive_grappling_loss_diff,l1y_defensive_grappling_loss_diff,l3y_defensive_grappling_loss_diff,l5y_defensive_grappling_loss_diff,all_overall_fighter_score_diff,l1y_overall_fighter_score_diff,l3y_overall_fighter_score_diff,l5y_overall_fighter_score_diff,age_sum
4,0,-0.145106,-4.0,-3.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.780915,0.786758,0.793333,0.786758,0.786758,-14.047712,8.431111,-14.047712,-14.047712,55.975359
8,1,-1.927447,-3.0,-7.0,2.0,-1.0,2.0,2.0,0.0,-1.0,...,0.888132,-3.640309,-4.028804,-3.640309,-3.640309,32.696678,-35.959590,32.696678,32.696678,55.162218
10,1,-4.355921,1.0,2.0,4.0,-1.0,0.0,3.0,0.0,0.0,...,3.036812,8.436528,3.218139,5.455113,8.445653,21.635663,41.571406,19.575704,18.427454,61.785079
12,1,3.008898,1.0,1.0,3.0,1.0,3.0,3.0,2.0,1.0,...,0.553801,0.640272,0.488809,0.640272,0.640272,8.434783,7.829006,8.434783,8.434783,59.030801
14,1,-3.189596,-1.0,-3.0,2.0,1.0,2.0,2.0,0.0,0.0,...,8.246001,0.674021,0.190722,0.674021,0.674021,-9.523442,22.528351,-9.523442,-9.523442,47.208761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9854,1,-0.643395,4.0,7.0,11.0,-1.0,1.0,3.0,2.0,-1.0,...,3.098993,15.366362,1.244329,3.160463,6.265374,5.574709,-11.560400,-1.549324,-14.147277,76.640657
9856,1,-5.065024,-2.0,0.0,4.0,0.0,0.0,2.0,-1.0,0.0,...,-2.824357,-0.694178,-0.184018,-0.715660,-0.612203,6.318019,10.398957,-11.965348,-3.380163,66.245038
9858,1,1.418207,0.0,-3.0,0.0,1.0,0.0,0.0,-1.0,0.0,...,-0.253515,0.119151,-0.068791,0.119151,0.119151,66.839385,113.577614,66.839385,66.839385,57.111567
9860,0,0.514716,-3.0,-5.0,0.0,-1.0,0.0,0.0,1.0,0.0,...,-2.952639,-2.798817,-0.065495,-2.798817,-2.798817,-6.275278,20.205072,-6.275278,-6.275278,64.974675


In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=91)

# Use the entire data set with all features as a baseline

In [26]:
all_features = X_train.columns.tolist()
# remove result from all_features
if 'result' in all_features:
    all_features.remove('result')
model_test_score(X_train, X_test, all_features, _max_iter = 20000, scaled=True)

Training set size: (3381, 321) accuracy: 0.6663708961845608
Test set size: (846, 321) accuracy: 0.6430260047281324
Test set neg log loss: -0.649790959567002. Probability to observe data given model: 0.5221549168431826


In [15]:
best_features = additive_greedy(X_train, X_test, search_doubles=True, _max_iter=30000)

Starting with an empty feature set.
Added single feature: l5y_overall_fighter_score_diff
Negative log loss on training set: -0.680011906474426
Training set size: (5052, 321) accuracy: 0.5696753760886778
Test set size: (1264, 321) accuracy: 0.5862341772151899
Test set neg log loss: -0.6793573544798297. Probability to observe data given model: 0.5069426721432919
Added single feature: age_diff
Negative log loss on training set: -0.6706129950769397
Training set size: (5052, 321) accuracy: 0.5892715756136183
Test set size: (1264, 321) accuracy: 0.6107594936708861
Test set neg log loss: -0.6674214747863464. Probability to observe data given model: 0.5130297338414997
Added single feature: l5y_wins_diff
Negative log loss on training set: -0.664536820846416
Training set size: (5052, 321) accuracy: 0.5894695170229612
Test set size: (1264, 321) accuracy: 0.6257911392405063
Test set neg log loss: -0.6615886817490836. Probability to observe data given model: 0.516030874107386
Added single feature: 

In [16]:
model_test_score(X_train, X_test, best_features, _max_iter = 30000, scaled=True)

Training set size: (5052, 321) accuracy: 0.6526128266033254
Test set size: (1264, 321) accuracy: 0.6416139240506329
Test set neg log loss: -0.6480221324476045. Probability to observe data given model: 0.52307933594839


In [17]:
best_features

['l5y_overall_fighter_score_diff',
 'age_diff',
 'l5y_wins_diff',
 'l5y_wins_wins_diff',
 'l1y_inf_clinch_strikes_accuracy_diff',
 'all_abs_head_strikes_accuracy_diff',
 'all_inf_distance_strikes_accuracy_diff',
 'l5y_inf_takedowns_attempts_per_min_diff',
 'all_inf_total_strikes_attempts_per_min_diff',
 'l1y_inf_clinch_strikes_landed_per_min_diff',
 'all_abs_leg_strikes_attempts_per_min_diff',
 'all_abs_takedowns_attempts_per_min_diff',
 'all_abs_control_per_min_diff',
 'l3y_abs_takedowns_landed_per_min_diff',
 'reach_diff',
 'all_inf_sub_attempts_per_min_diff',
 'l3y_defensive_grappling_loss_diff',
 'l3y_inf_sub_attempts_per_min_diff',
 'all_inf_clinch_strikes_accuracy_diff',
 'l5y_inf_body_strikes_accuracy_diff',
 'l1y_inf_body_strikes_landed_per_min_diff',
 'l3y_abs_body_strikes_accuracy_diff',
 'l5y_inf_knockdowns_per_min_diff',
 'l1y_abs_total_strikes_attempts_per_min_diff',
 'all_wins_wins_diff',
 'all_losses_losses_diff',
 'l1y_inf_ground_strikes_attempts_per_min_diff',
 'all_de

In [ ]:
best_features = ['l5y_overall_fighter_score_diff',
 'age_diff',
 'l5y_wins_diff',
 'l5y_wins_wins_diff',
 'l1y_inf_clinch_strikes_accuracy_diff',
 'all_abs_head_strikes_accuracy_diff',
 'all_inf_distance_strikes_accuracy_diff',
 'l5y_inf_takedowns_attempts_per_min_diff',
 'all_inf_total_strikes_attempts_per_min_diff',
 'l1y_inf_clinch_strikes_landed_per_min_diff',
 'all_abs_leg_strikes_attempts_per_min_diff',
 'all_abs_takedowns_attempts_per_min_diff',
 'all_abs_control_per_min_diff',
 'l3y_abs_takedowns_landed_per_min_diff',
 'reach_diff',
 'all_inf_sub_attempts_per_min_diff',
 'l3y_defensive_grappling_loss_diff',
 'l3y_inf_sub_attempts_per_min_diff',
 'all_inf_clinch_strikes_accuracy_diff',
 'l5y_inf_body_strikes_accuracy_diff',
 'l1y_inf_body_strikes_landed_per_min_diff',
 'l3y_abs_body_strikes_accuracy_diff',
 'l5y_inf_knockdowns_per_min_diff',
 'l1y_abs_total_strikes_attempts_per_min_diff',
 'all_wins_wins_diff',
 'all_losses_losses_diff',
 'l1y_inf_ground_strikes_attempts_per_min_diff',
 'all_defensive_grappling_loss_diff',
 'l3y_losses_dec_diff',
 'l5y_defensive_grappling_loss_diff',
 'l3y_overall_fighter_score_diff',
 'l1y_abs_ground_strikes_landed_per_min_diff',
 'all_inf_head_strikes_accuracy_diff',
 'l3y_inf_head_strikes_accuracy_diff',
 'l3y_inf_distance_strikes_landed_per_min_diff',
 'l3y_inf_sig_strikes_attempts_per_min_diff',
 'l3y_inf_body_strikes_accuracy_diff',
 'l3y_inf_takedowns_accuracy_diff',
 'l3y_abs_total_strikes_attempts_per_min_diff',
 'l3y_abs_control_per_min_diff',
 'l5y_fight_math_diff',
 'l1y_wins_sub_diff',
 'l1y_wins_ko_diff',
 'all_abs_sig_strikes_accuracy_diff',
 'l1y_abs_clinch_strikes_attempts_per_min_diff',
 'l1y_abs_clinch_strikes_landed_per_min_diff',
 'all_abs_clinch_strikes_accuracy_diff',
 'l3y_abs_leg_strikes_attempts_per_min_diff',
 'all_wins_ko_diff',
 'l3y_wins_ko_diff',
 'l1y_losses_losses_diff',
 'l1y_inf_body_strikes_attempts_per_min_diff',
 'l3y_abs_body_strikes_attempts_per_min_diff',
 'l3y_abs_body_strikes_landed_per_min_diff',
 'l1y_abs_body_strikes_attempts_per_min_diff',
 'l1y_inf_distance_strikes_accuracy_diff',
 'l1y_inf_total_strikes_accuracy_diff',
 'l5y_abs_body_strikes_landed_per_min_diff',
 'all_offensive_grappling_score_diff',
 'l3y_inf_takedowns_landed_per_min_diff',
 'l3y_offensive_grappling_score_diff']